# Automated Options Trading Monitor and Execution

This notebook automates the monitoring of option prices and places vertical spread orders when the conditions are met.

In [ ]:
import time
import logging
import datetime
import os
import sys

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger()

# Configuration
CYCLE_WAIT_SECONDS = 60
DB_PATH = '../database/option_strategies.db'
IBKR_HOST = '127.0.0.1'
IBKR_PORT = 7497

# Market closed settings
ALLOW_MARKET_CLOSED_ORDERS = True  # Set to True to allow orders when market is closed

# Print startup message
logger.info("=" * 80)
logger.info("Starting automated trading system")
logger.info("=" * 80)

# Import the modules
import price_monitor
import vertical_spread_order

cycle = 1
try:
    while True:  # Run indefinitely until user stops the process
        logger.info(f"Starting cycle {cycle}")
        
        # Step 1: Run price monitor
        logger.info("Starting price monitor...")
        try:
            # Run price monitor function
            price_monitor.monitor_prices(
                db_path=DB_PATH,
                ibkr_host=IBKR_HOST,
                ibkr_port=IBKR_PORT
            )
            logger.info("Price monitor completed successfully")
        except Exception as e:
            logger.error(f"Error running price monitor: {str(e)}")
        
        # Step 2: Place orders for triggered strategies
        logger.info("Starting order placement...")
        try:
            # Run trading app function
            vertical_spread_order.run_trading_app(
                db_path=DB_PATH,
                target_date=datetime.datetime.now().strftime('%Y-%m-%d'),
                ibkr_host=IBKR_HOST,
                ibkr_port=IBKR_PORT,
                allow_market_closed=ALLOW_MARKET_CLOSED_ORDERS
            )
            logger.info("Order placement completed successfully")
        except Exception as e:
            logger.error(f"Error placing orders: {str(e)}")
        
        # Wait before starting the next cycle
        logger.info(f"Waiting {CYCLE_WAIT_SECONDS} seconds until next cycle...")
        time.sleep(CYCLE_WAIT_SECONDS)
        
        # Increment cycle counter
        cycle += 1

except KeyboardInterrupt:
    logger.info("Automated trading system stopped by user")
except Exception as e:
    logger.error(f"Unexpected error stopped the automated trading system: {str(e)}")
finally:
    logger.info("Automated trading system completed")

2025-04-11 16:32:04,201 - INFO - ================================================================================
2025-04-11 16:32:04,203 - INFO - Starting automated trading system
2025-04-11 16:32:04,205 - INFO - ================================================================================
2025-04-11 16:32:04,859 - INFO - Starting cycle 1
2025-04-11 16:32:04,860 - INFO - Starting price monitor...
2025-04-11 16:32:04,861 - INFO - Connecting to IBKR at 127.0.0.1:7497
2025-04-11 16:32:04,882 - INFO - sent startApi
2025-04-11 16:32:04,891 - INFO - REQUEST startApi {}
2025-04-11 16:32:04,892 - INFO - SENDING startApi b'\x00\x00\x00\x0871\x002\x001\x00\x00'
2025-04-11 16:32:04,893 - INFO - ANSWER connectAck {}
2025-04-11 16:32:05,496 - INFO - disconnecting
2025-04-11 16:32:05,497 - INFO - ANSWER connectionClosed {}
2025-04-11 16:32:14,898 - ERROR - Failed to connect to IBKR within timeout
2025-04-11 16:32:14,898 - ERROR - Failed to connect to IBKR. Exiting.
2025-04-11 16:32:14,899 - INFO

In [2]:
os.path.join(os.getcwd(), 'price_monitor.py')

'/home/chris_s_dodd/source/optcom/monitor/price_monitor.py'

## Configure Market Closed Order Behavior

You can control whether orders are allowed to be placed when the market is closed by setting the `ALLOW_MARKET_CLOSED_ORDERS` variable at the top of the notebook.

- When `ALLOW_MARKET_CLOSED_ORDERS = True`: If the market is closed, orders will be placed using the estimated premium from the database.
- When `ALLOW_MARKET_CLOSED_ORDERS = False`: Orders will only be placed if the market is open AND the actual market premium is greater than or equal to the estimated premium.

This allows for more flexibility in strategy execution.